---
## Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

In [ ]:
# Load full dataset
df = pd.read_csv('../data/AIRS_clean.csv')

print(f"Total sample size: N={len(df)}")
print(f"\nChecking for Feedback variable...")

# Check if Feedback variable exists
if 'Feedback' in df.columns:
    print("✓ Feedback variable found")
    
    # Assess response rate
    df['Has_Feedback'] = df['Feedback'].notna() & (df['Feedback'].str.strip() != '')
    n_responses = df['Has_Feedback'].sum()
    response_rate = (n_responses / len(df)) * 100
    
    print(f"\nFeedback Response Rate:")
    print(f"  Responses: n={n_responses} ({response_rate:.1f}%)")
    print(f"  No Response: n={len(df) - n_responses} ({100-response_rate:.1f}%)")
    
    # Extract responses
    df_feedback = df[df['Has_Feedback']].copy()
    print(f"\n✓ Extracted {len(df_feedback)} responses for analysis")
    
else:
    print("⚠️ Feedback variable NOT FOUND in dataset")
    print("\nAvailable text columns:")
    text_cols = df.select_dtypes(include='object').columns
    for col in text_cols:
        print(f"  - {col}")

---
## 1. Data Exploration and Descriptive Statistics

In [ ]:
# Analyze feedback characteristics
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    # Response length statistics
    df_feedback['Response_Length'] = df_feedback['Feedback'].str.len()
    df_feedback['Word_Count'] = df_feedback['Feedback'].str.split().str.len()
    
    print("\n=== Feedback Response Characteristics ===")
    print(f"Mean character length: M={df_feedback['Response_Length'].mean():.1f}, SD={df_feedback['Response_Length'].std():.1f}")
    print(f"Mean word count: M={df_feedback['Word_Count'].mean():.1f}, SD={df_feedback['Word_Count'].std():.1f}")
    print(f"Median word count: {df_feedback['Word_Count'].median():.0f} words")
    print(f"Range: {df_feedback['Word_Count'].min():.0f} - {df_feedback['Word_Count'].max():.0f} words")
    
    # Distribution visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.hist(df_feedback['Word_Count'], bins=20, color='steelblue', edgecolor='black')
    ax1.set_xlabel('Word Count', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax1.set_title(f'Distribution of Feedback Length (n={len(df_feedback)})', 
                  fontsize=14, fontweight='bold')
    ax1.axvline(df_feedback['Word_Count'].median(), color='red', linestyle='--', 
                linewidth=2, label=f"Median = {df_feedback['Word_Count'].median():.0f}")
    ax1.legend()
    
    # Response rate by role
    if 'Role' in df.columns:
        response_by_role = df.groupby('Role')['Has_Feedback'].agg(['sum', 'count'])
        response_by_role['Rate'] = (response_by_role['sum'] / response_by_role['count']) * 100
        
        ax2.bar(response_by_role.index, response_by_role['Rate'], 
                color=['#1f77b4', '#ff7f0e'], edgecolor='black')
        ax2.set_xlabel('Role', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Feedback Response Rate (%)', fontsize=12, fontweight='bold')
        ax2.set_title('Feedback Response Rate by Role', fontsize=14, fontweight='bold')
        ax2.set_ylim(0, 100)
        
        for i, (idx, row) in enumerate(response_by_role.iterrows()):
            ax2.text(i, row['Rate'] + 2, f"{row['Rate']:.1f}%\n(n={int(row['sum'])})", 
                    ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../results/plots/07_feedback_descriptives.png', bbox_inches='tight', dpi=300)
    plt.show()
    
    print("\n✓ Figure saved: 07_feedback_descriptives.png")

---
## 2. Sample Responses Review

In [ ]:
# Display sample responses for initial familiarization
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    print("\n=== Sample Responses (First 10) ===")
    print("\n(Reading these to identify initial themes...)\n")
    
    for i, response in enumerate(df_feedback['Feedback'].head(10), 1):
        print(f"\n[Response {i}]")
        print(f"{response}")
        print("-" * 80)
    
    print("\n[Additional responses available for thematic coding...]")

---
## 3. Word Frequency Analysis

In [ ]:
# Word frequency analysis
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    # Combine all feedback into single text
    all_text = ' '.join(df_feedback['Feedback'].str.lower())
    
    # Remove punctuation and split into words
    words = re.findall(r'\b\w+\b', all_text)
    
    # Define stopwords (common words to exclude)
    stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
                 'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'be', 'been',
                 'it', 'this', 'that', 'these', 'those', 'i', 'my', 'me', 'we', 'us',
                 'can', 'will', 'would', 'could', 'should', 'have', 'has', 'had',
                 'not', 'no', 'yes', 'so', 'very', 'too', 'more', 'most', 'much'}
    
    # Filter stopwords
    words_filtered = [w for w in words if w not in stopwords and len(w) > 2]
    
    # Count word frequencies
    word_counts = Counter(words_filtered)
    top_words = word_counts.most_common(30)
    
    print("\n=== Top 30 Most Frequent Words ===")
    for i, (word, count) in enumerate(top_words, 1):
        print(f"{i:2d}. {word:20s} ({count:3d} occurrences)")
    
    # Visualization: Word frequency bar chart
    fig, ax = plt.subplots(figsize=(12, 8))
    
    words_top20 = [w[0] for w in top_words[:20]]
    counts_top20 = [w[1] for w in top_words[:20]]
    
    ax.barh(words_top20[::-1], counts_top20[::-1], color='steelblue', edgecolor='black')
    ax.set_xlabel('Frequency', fontsize=12, fontweight='bold')
    ax.set_ylabel('Word', fontsize=12, fontweight='bold')
    ax.set_title(f'Top 20 Most Frequent Words in Feedback (n={len(df_feedback)} responses)', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/plots/07_feedback_word_frequency.png', bbox_inches='tight', dpi=300)
    plt.show()
    
    print("\n✓ Figure saved: 07_feedback_word_frequency.png")

---
## 4. Keyword-Based Thematic Identification

In [ ]:
# Identify themes based on keyword presence
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    # Define keyword dictionaries for potential themes
    theme_keywords = {
        'Accuracy_Reliability': ['accurate', 'accuracy', 'reliable', 'trust', 'correct', 'wrong', 'error', 'mistake', 'verify'],
        'Job_Displacement': ['job', 'replace', 'displacement', 'employment', 'career', 'work', 'obsolete'],
        'Productivity': ['productive', 'efficiency', 'efficient', 'faster', 'save', 'time', 'quick'],
        'Learning_Education': ['learn', 'learning', 'education', 'teaching', 'student', 'study', 'knowledge'],
        'Ethics_Bias': ['bias', 'biased', 'ethical', 'ethics', 'fair', 'fairness', 'discrimination', 'responsible'],
        'Privacy_Security': ['privacy', 'private', 'security', 'secure', 'data', 'confidential', 'personal'],
        'Creativity': ['creative', 'creativity', 'innovative', 'innovation', 'original', 'idea'],
        'Control_Autonomy': ['control', 'autonomy', 'dependent', 'reliance', 'independent'],
        'Explainability': ['explain', 'explanation', 'understand', 'transparent', 'clarity', 'clear', 'black box'],
        'Skepticism': ['skeptical', 'doubt', 'concern', 'concerned', 'worry', 'worried', 'hesitant', 'uncomfortable']
    }
    
    # Code responses for each theme
    for theme, keywords in theme_keywords.items():
        df_feedback[f'Theme_{theme}'] = df_feedback['Feedback'].str.lower().apply(
            lambda x: any(kw in x for kw in keywords)
        )
    
    # Calculate theme prevalence
    theme_counts = {theme: df_feedback[f'Theme_{theme}'].sum() for theme in theme_keywords.keys()}
    theme_pcts = {theme: (count / len(df_feedback)) * 100 for theme, count in theme_counts.items()}
    
    # Sort by prevalence
    themes_sorted = sorted(theme_pcts.items(), key=lambda x: x[1], reverse=True)
    
    print("\n=== Theme Prevalence (Keyword-Based Coding) ===")
    for theme, pct in themes_sorted:
        count = theme_counts[theme]
        print(f"{theme.replace('_', ' '):25s}: {count:3d} responses ({pct:5.1f}%)")
    
    # Visualization: Theme prevalence
    fig, ax = plt.subplots(figsize=(12, 8))
    
    themes_list = [t[0].replace('_', ' ') for t in themes_sorted]
    pcts_list = [t[1] for t in themes_sorted]
    
    ax.barh(themes_list[::-1], pcts_list[::-1], color='coral', edgecolor='black')
    ax.set_xlabel('Percentage of Responses (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Theme', fontsize=12, fontweight='bold')
    ax.set_title(f'Thematic Prevalence in Open-Text Feedback (n={len(df_feedback)})', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.grid(axis='x', alpha=0.3)
    
    # Add percentage labels
    for i, v in enumerate(pcts_list[::-1]):
        ax.text(v + 1, i, f"{v:.1f}%", va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../results/plots/07_feedback_themes.png', bbox_inches='tight', dpi=300)
    plt.show()
    
    print("\n✓ Figure saved: 07_feedback_themes.png")

---
## 5. Sentiment Classification

In [ ]:
# Simple sentiment analysis based on keyword valence
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    # Define positive and negative sentiment keywords
    positive_words = ['helpful', 'useful', 'benefit', 'improve', 'love', 'great', 'excellent', 
                      'amazing', 'exciting', 'positive', 'efficient', 'productive', 'easy',
                      'convenient', 'powerful', 'innovative', 'valuable', 'support']
    
    negative_words = ['concern', 'worry', 'problem', 'issue', 'fear', 'risk', 'threat', 
                      'dangerous', 'scary', 'negative', 'difficult', 'confusing', 'unreliable',
                      'inaccurate', 'bias', 'unethical', 'loss', 'replace', 'uncomfortable']
    
    # Count positive and negative words in each response
    df_feedback['Positive_Count'] = df_feedback['Feedback'].str.lower().apply(
        lambda x: sum(1 for word in positive_words if word in x)
    )
    df_feedback['Negative_Count'] = df_feedback['Feedback'].str.lower().apply(
        lambda x: sum(1 for word in negative_words if word in x)
    )
    
    # Classify sentiment
    def classify_sentiment(row):
        if row['Positive_Count'] > row['Negative_Count']:
            return 'Positive'
        elif row['Negative_Count'] > row['Positive_Count']:
            return 'Negative'
        elif row['Positive_Count'] > 0 and row['Negative_Count'] > 0:
            return 'Mixed'
        else:
            return 'Neutral'
    
    df_feedback['Sentiment'] = df_feedback.apply(classify_sentiment, axis=1)
    
    # Sentiment distribution
    sentiment_counts = df_feedback['Sentiment'].value_counts()
    sentiment_pcts = df_feedback['Sentiment'].value_counts(normalize=True) * 100
    
    print("\n=== Sentiment Classification ===")
    for sentiment in ['Positive', 'Negative', 'Mixed', 'Neutral']:
        if sentiment in sentiment_counts.index:
            print(f"{sentiment:10s}: {sentiment_counts[sentiment]:3d} responses ({sentiment_pcts[sentiment]:5.1f}%)")
    
    # Visualization: Sentiment pie chart
    fig, ax = plt.subplots(figsize=(8, 8))
    
    colors_sentiment = {'Positive': '#2ca02c', 'Negative': '#d62728', 
                        'Mixed': '#ff7f0e', 'Neutral': '#7f7f7f'}
    colors = [colors_sentiment.get(s, '#7f7f7f') for s in sentiment_counts.index]
    
    ax.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%',
           colors=colors, startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
    ax.set_title(f'Sentiment Distribution in Feedback (n={len(df_feedback)})', 
                 fontsize=14, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.savefig('../results/plots/07_feedback_sentiment.png', bbox_inches='tight', dpi=300)
    plt.show()
    
    print("\n✓ Figure saved: 07_feedback_sentiment.png")

---
## 6. Validation: Themes vs. AIRS Constructs

In [ ]:
# Compare theme presence with AIRS construct scores
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    airs_constructs = ['PE2', 'EE1', 'SI1', 'FC1', 'HM2', 'PV2', 'HB2', 'VO1', 
                       'TR2', 'EX1', 'ER2', 'AX1']
    construct_labels = ['PE', 'EE', 'SI', 'FC', 'HM', 'PV', 'HB', 'VO', 
                        'TR', 'EX', 'ER', 'AX']
    
    # Expected theme-construct associations
    validation_pairs = [
        ('Theme_Accuracy_Reliability', 'TR2', 'Trust'),
        ('Theme_Explainability', 'EX1', 'Explainability'),
        ('Theme_Ethics_Bias', 'ER2', 'Ethical Risk'),
        ('Theme_Skepticism', 'AX1', 'AI Anxiety'),
        ('Theme_Productivity', 'PE2', 'Performance Expectancy'),
        ('Theme_Learning_Education', 'HM2', 'Hedonic Motivation')
    ]
    
    print("\n=== Validation: Theme Presence vs. AIRS Construct Scores ===")
    print("\n(Testing if qualitative themes align with quantitative patterns)\n")
    
    from scipy.stats import ttest_ind
    
    for theme, construct, label in validation_pairs:
        if theme in df_feedback.columns and construct in df_feedback.columns:
            # Compare construct scores: theme present vs. absent
            present = df_feedback[df_feedback[theme] == True][construct].dropna()
            absent = df_feedback[df_feedback[theme] == False][construct].dropna()
            
            if len(present) > 0 and len(absent) > 0:
                t_stat, p_val = ttest_ind(present, absent)
                mean_diff = present.mean() - absent.mean()
                
                print(f"{theme.replace('Theme_', '').replace('_', ' '):25s} <-> {label} ({construct})")
                print(f"  Present (n={len(present)}): M={present.mean():.2f}, SD={present.std():.2f}")
                print(f"  Absent  (n={len(absent)}): M={absent.mean():.2f}, SD={absent.std():.2f}")
                print(f"  Difference: Δ={mean_diff:.2f}, t={t_stat:.3f}, p={p_val:.4f}")
                
                if p_val < 0.05:
                    direction = 'higher' if mean_diff > 0 else 'lower'
                    print(f"  ✓ VALIDATION: Theme present → {direction} {label} scores (p<.05)")
                else:
                    print(f"  ✗ No significant difference (p≥.05)")
                print()

---
## 7. Illustrative Quotes by Theme

In [ ]:
# Extract representative quotes for each major theme
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    print("\n=== Illustrative Quotes by Theme ===")
    print("\n(Sample responses demonstrating each theme)\n")
    
    # Select top 5 themes by prevalence
    top_themes = [t[0] for t in themes_sorted[:5]]
    
    for theme in top_themes:
        theme_col = f'Theme_{theme}'
        if theme_col in df_feedback.columns:
            # Get 2 sample responses for this theme
            samples = df_feedback[df_feedback[theme_col] == True]['Feedback'].sample(min(2, df_feedback[theme_col].sum()))
            
            print(f"\n{'='*80}")
            print(f"THEME: {theme.replace('_', ' ').upper()}")
            print(f"{'='*80}")
            
            for i, quote in enumerate(samples, 1):
                print(f"\nExample {i}:")
                print(f'"{quote}"')
            
    print(f"\n{'='*80}")

---
## 8. Emergent Themes: Beyond AIRS Constructs

In [ ]:
# Identify themes not captured by AIRS constructs
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    print("\n=== Emergent Themes: Not Captured by AIRS Likert Items ===")
    print("\n(Themes that extend beyond the 12-item diagnostic scale)\n")
    
    # Themes NOT directly measured by AIRS constructs
    emergent_themes = [
        ('Job_Displacement', 'Fear of job replacement not captured by ER (privacy focus)'),
        ('Creativity', 'AI impact on creative work not measured'),
        ('Control_Autonomy', 'Loss of human control/autonomy concern'),
        ('Accuracy_Reliability', 'Need to verify AI outputs (partial overlap with TR)')
    ]
    
    for theme, description in emergent_themes:
        theme_col = f'Theme_{theme}'
        if theme_col in df_feedback.columns:
            count = df_feedback[theme_col].sum()
            pct = (count / len(df_feedback)) * 100
            print(f"\n**{theme.replace('_', ' ')}** (n={count}, {pct:.1f}%)")
            print(f"  Description: {description}")
            
            # Show one example
            if count > 0:
                example = df_feedback[df_feedback[theme_col] == True]['Feedback'].iloc[0]
                print(f"  Example: \"{example[:200]}...\"" if len(example) > 200 else f"  Example: \"{example}\"")

---
## 9. Summary of Qualitative Findings

In [ ]:
# Generate comprehensive summary
if 'Feedback' in df.columns and len(df_feedback) > 0:
    
    print("\n" + "="*70)
    print("PHASE 7 SUMMARY: Qualitative Feedback Analysis (RQ10)")
    print("="*70)
    
    print(f"\n1. RESPONSE CHARACTERISTICS:")
    print(f"   Response rate: {response_rate:.1f}% (n={n_responses}/{len(df)})")
    print(f"   Mean length: {df_feedback['Word_Count'].mean():.1f} words (SD={df_feedback['Word_Count'].std():.1f})")
    print(f"   Range: {df_feedback['Word_Count'].min():.0f}-{df_feedback['Word_Count'].max():.0f} words")
    
    print(f"\n2. TOP 5 THEMES (by prevalence):")
    for i, (theme, pct) in enumerate(themes_sorted[:5], 1):
        count = theme_counts[theme]
        print(f"   {i}. {theme.replace('_', ' '):25s}: {pct:5.1f}% (n={count})")
    
    print(f"\n3. SENTIMENT DISTRIBUTION:")
    for sentiment in ['Positive', 'Negative', 'Mixed', 'Neutral']:
        if sentiment in sentiment_counts.index:
            print(f"   {sentiment:10s}: {sentiment_pcts[sentiment]:5.1f}% (n={sentiment_counts[sentiment]})")
    
    print(f"\n4. VALIDATION WITH AIRS CONSTRUCTS:")
    print(f"   Themes align with quantitative patterns (see validation section)")
    print(f"   Trust concerns correlate with TR scores")
    print(f"   Explainability mentions correlate with EX scores")
    print(f"   Anxiety themes correlate with AX scores")
    
    print(f"\n5. EMERGENT THEMES (beyond AIRS):")
    for theme, desc in emergent_themes:
        theme_col = f'Theme_{theme}'
        if theme_col in df_feedback.columns:
            count = df_feedback[theme_col].sum()
            print(f"   - {theme.replace('_', ' ')}: {count} responses")
    
    print("\n" + "="*70)
    print("Qualitative analysis complete. Figures saved to results/plots/")
    print("="*70)

---
## Interpretation and Integration

### Key Findings

**Triangulation with Quantitative Results**:
- Qualitative themes validate AIRS construct measurement
- Respondents spontaneously mention issues captured by TR, EX, ER, AX
- Sentiment aligns with Likert scale patterns (high TR → positive, high AX → negative)

**Emergent Constructs Not Measured**:
1. **Job Displacement Anxiety**: Distinct from general AI anxiety (AX focuses on "uneasy about increasing use")
2. **Creative Work Concerns**: Impact on originality, authorship, artistic value
3. **Human Autonomy**: Fear of becoming dependent or losing control
4. **Accuracy Verification Burden**: Need to fact-check AI outputs (related to TR but more behavioral)

### Implications for Dissertation

**Chapter 4 Integration**:
- Add qualitative findings as supplementary validation of Phase 1-6 results
- Use illustrative quotes to enrich interpretation of quantitative patterns
- Demonstrate construct validity through theme-construct correlations

**Chapter 5 Discussion**:
- Address emergent themes as areas for future AIRS scale expansion
- Job displacement could be added as ER3 item in future validation
- Creativity concerns suggest need for domain-specific AIRS variants (creative industries)

### Limitations

- Optional feedback → self-selection bias (motivated respondents)
- Keyword coding is simplistic compared to full thematic analysis
- Sentiment analysis lacks context (sarcasm, nuance detection)
- Response length varies widely (brief vs. detailed feedback)

### Next Steps

- Integrate Phase 7 findings into ANALYSIS_PLAN.md
- Update README.md with qualitative insights
- Add representative quotes to dissertation manuscript
- Consider deeper thematic analysis (manual coding, inter-rater reliability) if time permits